In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [3]:
def compute_fama_residuals_yearly(stock_df, factor_df, rf_col='無風險利率'):
    """
    stock_df: 股票日報酬率（index=日期，columns=股票）
    factor_df: Fama 因子與無風險利率（index=日期）
    rf_col: 無風險利率欄位名稱
    """
    # 計算超額報酬
    stock_df_excess = stock_df.sub(factor_df[rf_col], axis=0)

    # 選擇五個 Fama 因子
    factor_cols = ['市場風險溢酬', '規模溢酬(5因子)', '淨值市價比溢酬', '益本比溢酬', '股利殖利率溢酬']
    X_factors_all = factor_df[factor_cols]

    residuals = pd.DataFrame(index=stock_df.index, columns=stock_df.columns)

    # 以年份分群
    years = stock_df.index.year.unique()

    for year in years:
        idx = stock_df.index.year == year

        for stock in stock_df.columns:
            y = stock_df_excess.loc[idx, stock].values
            X = X_factors_all.loc[idx].values

            # 資料不足就跳過
            if len(y) < 10 or np.isnan(y).any() or np.isnan(X).any():
                continue

            model = LinearRegression().fit(X, y)
            pred = model.predict(X)
            residuals.loc[idx, stock] = y - pred

    return residuals

In [4]:
# 讀取股票報酬率資料（修正編碼亂碼問題，如遇亂碼可試 'big5', 'utf-8-sig'）
df = pd.read_csv("2009_2025_combinedreturn.csv", encoding='utf-8', header=0)

# 重新命名欄位（根據你給的樣本）
df.columns = ['代號', '名稱', '年月日', '日報酬率 %']

# 日期轉為 datetime 格式，並排序
df['年月日'] = pd.to_datetime(df['年月日'])
df = df.sort_values(['年月日', '代號'])

# 建立「代號 名稱」作為列名
df['股票'] = df['代號'].astype(str) + ' ' + df['名稱']

# pivot 成 [日期 × 股票] 的報酬率表格
pivot_df = df.pivot(index='年月日', columns='股票', values='日報酬率 %')

# 將報酬率轉為小數（% → 小數）
pivot_df = pivot_df / 100

In [5]:
pivot_df

股票,1101 台泥,1102 亞泥,1103 嘉泥,1104 環泥,1108 幸福,1109 信大,1110 東泥,1201 味全,1203 味王,1210 大成,...,9939 宏全,9940 信義,9941 裕融,9942 茂順,9943 好樂迪,9944 新麗,9945 潤泰新,9946 三發地產,9955 佳龍,9958 世紀鋼
年月日,,,,,,,,,,,,,,,,,,,,,
2009-01-05,0.025974,-0.008787,0.003922,0.015686,0.008333,0.011236,0.001087,-0.009091,0.016667,0.031746,...,0.042955,0.049587,0.000000,-0.004008,-0.011905,0.003690,0.045977,0.015873,0.019737,0.056604
2009-01-06,-0.007233,-0.003546,-0.003906,0.000000,-0.004959,0.000000,-0.010858,0.000000,0.008197,0.038462,...,0.037891,0.003937,0.009317,-0.014085,0.004016,0.007353,-0.016484,0.000000,-0.016129,0.069643
2009-01-07,0.023679,0.069395,0.066667,0.038610,0.009967,0.012346,0.013172,0.045872,0.004065,0.024074,...,0.034921,0.050980,0.009231,0.004082,-0.028000,0.000000,0.008380,0.000000,0.036066,0.068447
2009-01-08,-0.032028,-0.064892,-0.066176,-0.052045,-0.062500,-0.029268,-0.046587,-0.054825,-0.040486,-0.068716,...,-0.042945,-0.067164,-0.015244,-0.012195,-0.020576,-0.014599,-0.024931,0.000000,-0.066456,0.068750
2009-01-09,-0.025735,-0.010676,0.011811,0.000000,0.017544,0.001256,-0.003409,0.020882,0.025316,0.029126,...,0.048077,0.020000,0.003096,-0.008230,0.000000,0.003704,0.031250,0.045313,0.013559,0.008772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-20,0.011745,0.020362,0.003155,0.005102,0.003279,0.014577,0.029777,0.002950,0.014742,0.014308,...,-0.006536,-0.003559,0.000000,0.004237,0.002567,-0.005814,0.004926,0.015075,-0.001718,0.002786
2025-05-21,0.004975,-0.006652,0.015723,0.008460,-0.003268,0.002874,0.004819,0.005882,-0.002421,0.000000,...,0.000000,0.001786,0.008696,0.004219,0.008963,0.002924,0.011438,0.029703,0.015491,-0.002778
2025-05-22,-0.009901,-0.032366,-0.009288,-0.003356,0.003279,-0.005731,0.004796,-0.002924,-0.003641,0.001567,...,-0.003289,-0.003565,-0.017241,-0.008403,-0.002538,-0.002915,-0.022617,-0.009615,0.016949,0.005571


In [54]:
factor_df = pd.read_csv("2009_2025_factor.csv")

In [55]:
# 確保 pivot_df 與 factor_df 的 index 都是 datetime 且有排序
pivot_df = pivot_df.sort_index()
factor_df = factor_df.copy()
factor_df['年月日'] = pd.to_datetime(factor_df['年月日'])
factor_df = factor_df.set_index('年月日').sort_index()

# 取交集日期，進行對齊
common_dates = pivot_df.index.intersection(factor_df.index)
pivot_df = pivot_df.loc[common_dates]
factor_df = factor_df.loc[common_dates]

print("對齊後 pivot_df 日期數：", len(pivot_df))
print("對齊後 factor_df 日期數：", len(factor_df))

對齊後 pivot_df 日期數： 4024
對齊後 factor_df 日期數： 4024


In [56]:
factor_df

,市場風險溢酬,規模溢酬(5因子),淨值市價比溢酬,益本比溢酬,股利殖利率溢酬,動能因子,無風險利率,年化RF
年月日,,,,,,,,
2009-01-05,2.2358,-0.7279,-0.7123,-0.0963,-1.2563,-2.4933,1.42,0.005635
2009-01-06,0.5810,0.4534,-0.1311,0.5389,1.0736,-1.7522,1.42,0.005635
2009-01-07,1.2597,-0.0642,-0.2878,0.7035,1.6238,-1.7966,1.42,0.005635
2009-01-08,-5.4787,1.6974,-0.0881,0.0166,0.9078,1.6336,1.42,0.005635
2009-01-09,-0.7657,1.1987,0.6246,-0.5394,0.3964,0.2025,0.92,0.003651
...,...,...,...,...,...,...,...,...
2025-05-20,0.0004,-0.0735,0.4674,-0.0533,0.5828,-0.2350,1.70,0.006746
2025-05-21,1.2581,-0.1596,-0.6260,0.0305,-0.0156,-0.0221,1.70,0.006746
2025-05-22,-0.6163,0.1024,0.3821,0.3305,-0.0297,0.4201,1.70,0.006746


In [57]:
# 計算殘差序列
residual_df = compute_fama_residuals_yearly(pivot_df, factor_df, rf_col='年化RF')

# 儲存結果（可選）
residual_df.to_csv("2009_2025_fama_residual.csv")